<a href="https://colab.research.google.com/github/pawansaxena108/watson_coursera_capstone/blob/master/loan_defaulter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

--2020-12-07 06:58:13--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0s      

2020-12-07 06:58:13 (212 MB/s) - ‘loan_train.csv’ saved [23101/23101]



In [7]:
!pip install feature_engine

In [8]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder,LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from feature_engine import categorical_encoders as ce
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [9]:
data=pd.read_csv('loan_train.csv')

In [37]:
data.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


In [ ]:
data.info()

In [40]:
data.loan_status.value_counts()

PAIDOFF       260
COLLECTION     86
Name: loan_status, dtype: int64

In [10]:
x=data[set(data.columns)-{'loan_status'}]
y=data['loan_status']

In [7]:
data.describe(include=object)

,loan_status,effective_date,due_date,education,Gender
count,346,346,346,346,346
unique,2,7,23,4,2
top,PAIDOFF,9/11/2016,10/10/2016,High School or Below,male
freq,260,166,90,151,294


In [6]:
data.describe()

,Unnamed: 0,Unnamed: 0.1,Principal,terms,age
count,346.000000,346.000000,346.000000,346.000000,346.000000
mean,202.167630,202.167630,943.641618,22.653179,30.939306
std,115.459715,115.459715,109.425530,7.991006,6.039418
min,0.000000,0.000000,300.000000,7.000000,18.000000
25%,107.250000,107.250000,900.000000,15.000000,27.000000
50%,204.500000,204.500000,1000.000000,30.000000,30.000000
75%,298.750000,298.750000,1000.000000,30.000000,35.000000
max,399.000000,399.000000,1000.000000,30.000000,51.000000


In [11]:
num_pipeline=make_pipeline(SimpleImputer(strategy='mean'),StandardScaler())

In [12]:
cat_pipeline=make_pipeline(ce.OrdinalCategoricalEncoder())

In [13]:
label_encoder=LabelEncoder()
y=pd.Series(label_encoder.fit_transform(y))

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2)

In [15]:
c_transformer= ColumnTransformer([('num_pipeline',num_pipeline,make_column_selector(dtype_exclude='object')),('cat_pipeline',cat_pipeline,make_column_selector(dtype_include=object))])

In [16]:
x=c_transformer.fit_transform(x,y)

In [32]:
scores

,f1,jaccard,logloss
Decision Tree,NaN,NaN,NaN
KNN,NaN,NaN,NaN
Logistic,NaN,NaN,NaN


In [17]:
model_DT=DecisionTreeClassifier()
model_KNN=KNeighborsClassifier()
model_SVM= SVC()
model_LR=LogisticRegression() 

In [30]:
scores=pd.DataFrame(data=None,index=['Decision Tree','KNN','Logistic'],columns=['f1','jaccard','logloss'])

In [18]:
from sklearn.metrics import jaccard_score,f1_score,make_scorer,log_loss
from sklearn.model_selection import cross_validate

In [40]:
model_DT.fit(x,y)
model_KNN.fit(x,y)
model_LR.fit(x,y)
model_SVM.fit(x,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [39]:
scores.loc['Decision Tree','logloss']=log_loss(y,model_DT.predict_proba(x))
scores.loc['Decision Tree','f1']=f1_score(y,model_DT.predict(x))
scores.loc['Decision Tree','jaccard']=jaccard_score(y,model_DT.predict(x))

In [41]:
scores.loc['KNN','logloss']=log_loss(y,model_KNN.predict_proba(x))
scores.loc['KNN','f1']=f1_score(y,model_KNN.predict(x))
scores.loc['KNN','jaccard']=jaccard_score(y,model_KNN.predict(x))

In [47]:
scores.loc['Logistic','logloss']=log_loss(y,model_LR.predict_proba(x))
scores.loc['Logistic','f1']=f1_score(y,model_LR.predict(x))
scores.loc['Logistic','jaccard']=jaccard_score(y,model_LR.predict(x))

In [45]:
#scores.loc['SVM','logloss']=log_loss(y,model_SVM.predict_proba(x)) 

scores.loc['SVM','logloss']='predict prop not available in svm'
scores.loc['SVM','f1']=f1_score(y,model_SVM.predict(x))

scores.loc['SVM','jaccard']=jaccard_score(y,model_SVM.predict(x))

In [52]:
#final score
scores

,f1,jaccard,logloss
Decision Tree,1,1,9.99201e-16
KNN,0.994264,0.988593,0.0127143
Logistic,1,1,0.00838912
SVM,1,1,predict prop not available in svm


In [ ]:
#best value of K
5